<h1 style="text-align: center;"> FINAL PROJECT </h1>
<h2 style="text-align: center;"> Dự đoán điểm IT001 từ dữ liệu làm bài trên WECODE </h2>

---

#### Lớp: CS114.P21

#### Thành viên nhóm

1. Nguyễn Chí Cường - 23520199
2. Hứa Mạnh Tân - 23521396
3. Nguyễn Tấn Tài - 23521376

#### MỤC LỤC

---

## 1. Chuẩn bị dữ liệu và một số thông tin về bộ dữ liệu

### 1.1. Import thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from lazypredict.Supervised import LazyRegressor
from sklearn.linear_model import LinearRegression

### 1.2. Load bộ dữ liệu

In [2]:
# Đường dẫn đến các tập dữ liệu
WECODE_DATASET = "../dataset/wecode/annonimized.csv"
QT_DATASET = "../dataset/wecode/qt-public.csv"
TH_DATASET = "../dataset/wecode/th-public.csv"
CK_DATASET = "../dataset/wecode/ck-public.csv"

In [3]:
wecode_df = pd.read_csv(WECODE_DATASET).rename(
    columns={
        "concat('it001',`assignment_id`)": "assignment_id",
        "concat('it001',`problem_id`)": "problem_id",
        "concat('it001', username)": "username",
        "concat('it001',`language_id`)": "language_id",
    }
)

qt_df = pd.read_csv(QT_DATASET).rename(columns={"hash": "username", "diemqt": "QT"})
th_df = pd.read_csv(TH_DATASET).rename(columns={"hash": "username"})
ck_df = pd.read_csv(CK_DATASET).rename(columns={"hash": "username"})

### 1.3. Thông tin bộ dữ liệu

#### 1.3.1. Thông tin bộ dữ liệu làm bài tập Wecode của sinh viên

In [4]:
wecode_df.head(10)

In [5]:
wecode_df.info()

In [6]:
wecode_df.nunique()

Từ thông tin trên thì ta có thể nhận thấy ngay bộ dữ liệu không hề có cột nào bị bỏ trống (tất cả các cột đều có 295198 giá trị là non-null), nên ta không cần phải xử lí dữ liệu khuyết thiết cho dữ liệu này. 

Những ta dễ thấy, các đặc trưng của dữ liệu này không mang giá trị đánh giá chung về khả năng của sinh viên, mà nó chỉ chứa thông tin về những lần nộp bài wecode của sinh viên một các rời rạc, lẻ tẻ. Vì vậy, chúng cần phải được tổng hợp lại và thay thế bằng những đặc trưng mới có giá trị đánh giá hoạt động học tập của sinh viên (ví dụ: tổng điểm của các bài wecode, số lần thử nộp bài, số bài đã giải, ...). Và từ đó ta có thể xây dựng các mô hình dự đoán điểm quá trình, thực hành và cuối kì dựa trên các đặc trưng mới này hiệu quả hơn.

#### 1.3.2. Thông tin bộ dữ liệu điểm của sinh viên

In [7]:
qt_df.head(10)

In [8]:
qt_df.info()

In [9]:
qt_df.nunique()

In [10]:
th_df.head(10)

In [11]:
th_df.info()

In [12]:
th_df.nunique()

In [13]:
ck_df.head(10)

In [14]:
ck_df.info()

In [15]:
ck_df.nunique()

Từ thông tin của 3 dataframe điểm trên, ta có thể thấy những thông tin sau:
- Cả 3 dataframe đều có 761 `username` non-null và khác nhau
- Cả 3 dataframe có cột điểm chỉ có 755/761 giá trị là non-null

In [16]:
print(
    "Number of similar usernames between qt_df and th_df: "
    + str(sum(qt_df["username"] == th_df["username"]))
)
print(
    "Number of similar usernames between qt_df and ck_df: "
    + str(sum(qt_df["username"] == ck_df["username"]))
)

Nhận thấy cả 3 dataframe chứa điểm quá trình, thực hành, cuối kì của cùng 761 sinh viên, vậy nên ta có thể hợp nhất cả 3 dataframe được load riêng lẻ thành 1 dataframe duy nhất dể tiện hơn trong việc thao tác. Và sẵn tiện ta có thể tính toán luôn cột `TBTL` từ 3 cột loại điểm trên

In [17]:
# Hợp nhất điểm từ 3 dataframe thành 1 dataframe score_df duy nhất
score_df = pd.merge(
    pd.merge(qt_df, th_df, on="username", how="inner"),
    ck_df,
    on="username",
    how="inner",
)
score_df["QT"] = pd.to_numeric(score_df["QT"], errors="coerce")
score_df["TH"] = pd.to_numeric(score_df["TH"], errors="coerce")
score_df["CK"] = pd.to_numeric(score_df["CK"], errors="coerce")

### 1.4. Xử lí dữ liệu khuyết thiếu

Đối với `wecode_df` thì hoàn toàn không hề có dữ liệu khuyết thiếu. Bây giờ, ta xem xét những hàng bị bỏ trống (NaN) trong `score_df` để thực hiện lọc hoặc lấp đầy chúng, tránh lỗi trong quá trình thao tác.

In [18]:
score_df[(score_df["QT"].isna()) | (score_df["TH"].isna()) | (score_df["CK"].isna())]

Ta dễ dàng nhận thấy rằng chỉ có 6 `username` có ít nhất 1 trong 3 cột bị bỏ trống (NaN), và thực chất là cả 3 cột đều bị bỏ trống. Vì vậy ta có thể loại những `username` này ra khỏi dataframe `score_df`.

In [19]:
# Xử lý lọc dữ liệu không hợp lệ
score_df = score_df.dropna()

Sau khi loại bỏ NaN thì kết quả dataframe `score_df` thu được chứa 755 dòng đều non-null:

In [20]:
score_df.info()

## 2. Trích xuất đặc trưng và tiền xử lí dữ liệu mới

### 2.1. Trích xuất đặc trưng từ dữ liệu gốc

Như đã phân tích ở phần 1 thì tập dữ liệu wecode hiện tại không thể được dùng trực tiếp để đánh giá năng lực học tập của sinh viên. Nó là một bản ghi log thông tin các lần nộp bài của sinh viên trên wecode. Do đó, cần phải tổng hợp các đặc trưng hiện tại và trích xuất ra thành các đặt trưng mới có giá trị đánh giá năng lực học tập của sinh viên hơn.

Từ tập dữ liệu wecode trên, một số đặc trưng mới có thể trích xuất từ các đặc trưng hiện tại có thể kể đến đó là:
- Tổng số lần nộp bài (`submissions`)
- Tổng số lần thử đúng (`correct_submissions`)
- Tổng số vấn đề đã thử giải (`tried_problems`)
- Tổng số vấn đề đã giải (`solved_problems`)
- Tổng điểm của tất cả các lần nộp (`total_submissions_score`)
- Tổng điểm cuối cùng (`total_final_score`)

In [21]:
wecode_df["coef_score"] = wecode_df["pre_score"] * wecode_df["coefficient"] / 100

wecode_df["final_score"] = wecode_df["is_final"] * wecode_df["coef_score"]

wecode_df["is_final_solution"] = wecode_df["is_final"] * (
    wecode_df["pre_score"] == 10000
)

extracted_wecode_df = (
    wecode_df.groupby("username")
    .agg(
        submissions=("username", "size"),
        correct_submissions=("pre_score", lambda x: sum(x == 10000)),
        tried_problems=("is_final", "sum"),
        solved_problems=("is_final_solution", "sum"),
        total_submissions_score=("coef_score", "sum"),
        total_final_score=(
            "final_score",
            "sum",
        ),
    )
    .reset_index()
)

extracted_wecode_df.head(10)

Thế nhưng, những đặc trưng đó chưa đủ để đánh giá năng lực của sinh viên, nên ta cần phải biến đổi thêm một lần nữa để thu về những đặc trưng có giá trị đánh giá tốt hơn:
- Trung bình số lần thử trên mỗi vấn đề (`submissions/problem`)
- Tỉ lệ đúng của mỗi lần thử (`accuracy`)
- Trung bình điểm trên mỗi vấn đề (`score/problem`)
- Trung bình điểm trên mỗi lần thử (`score/submission`)

In [22]:
extracted_wecode_df["submissions/problem"] = (
    extracted_wecode_df["submissions"] / extracted_wecode_df["tried_problems"]
)
extracted_wecode_df["accuracy"] = (
    extracted_wecode_df["correct_submissions"] / extracted_wecode_df["submissions"]
)
extracted_wecode_df["score/problem"] = (
    extracted_wecode_df["total_final_score"] / extracted_wecode_df["tried_problems"]
)
extracted_wecode_df["score/submission"] = (
    extracted_wecode_df["total_submissions_score"] / extracted_wecode_df["submissions"]
)
extracted_wecode_df.head(10)

### 2.2. Xử lí dữ liệu khuyết thiếu

In [23]:
extracted_wecode_df.dropna(inplace=True)

### 2.3. Xử lí chuẩn hóa dữ liệu

In [24]:
# scaler = StandardScaler()

# for col in extracted_wecode_df.columns[1:]:
#     extracted_wecode_df[col] = scaler.fit_transform(extracted_wecode_df[[col]])

### 2.3. Hợp nhất tập dữ liệu điểm và tập dữ liệu wecode để hình thành tập huấn luyện cho mô hình

In [25]:
data_df = pd.merge(
    extracted_wecode_df,
    score_df,
    on="username",
    how="inner",
)
data_df.head(10)

In [26]:
data_df.info()

## 3. Phân tích dữ liệu

### Tính toán correlation

In [27]:
feature_cols = data_df.columns[1:-4]
point_cols = data_df.columns[-4:]

correlation_matrix = (
    data_df[feature_cols].corrwith(data_df[point_cols[0]]).to_frame(name=point_cols[0])
)
for point_col in point_cols[1:]:
    correlation_matrix[point_col] = data_df[feature_cols].corrwith(data_df[point_col])


print("Correlation matrix:")
print(correlation_matrix)

correlation_matrix.plot(
    kind="bar", figsize=(10, 6), title="Correlation between Features and Points"
)
plt.xlabel("Features")
plt.ylabel("Correlation")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Từ kết quả về hệ số tương quan Pearson (Pearson correlation) giữa các đặc trưng và các loại điểm, ta có thể thấy được ngay
- Hệ số tương quan giữa 6 đặc trưng đầu với điểm TH có giá trị tuyệt đối > 0.3. Do đó, có thể dùng mô hình hồi quy tuyến tính (linear regression) với 6 đặc trưng đầu để đánh giá điểm TH
- Các hệ số tương quan còn lại có giá trị tuyến đối khá nhỏ (< 0.2), vậy nên cần tìm 1 không tuyến tính để dự đoán điểm dựa trên các đặc trưng hiện có

In [28]:
fig, axes = plt.subplots(
    len(feature_cols), len(point_cols), figsize=(15, 30), sharey=True
)

for i, feature_col in enumerate(feature_cols):
    for j, point_col in enumerate(point_cols):
        axes[i, j].scatter(data_df[feature_col], data_df[point_col], alpha=0.5)
        axes[i, j].set_title(f"{feature_col} vs {point_col}")
        axes[i, j].set_xlabel(feature_col)
        axes[i, j].set_ylabel(point_col)

plt.tight_layout()
plt.show()

## 4. Huấn luyện mô hình dự đoán

In [29]:
OUTPUT_DIR = "../output/wecode/"

### 4.1. Huấn luyện mô hình dự đoán điểm quá trình (QT)

In [ ]:
qt_feature_cols = feature_cols

X = data_df[qt_feature_cols]
y = data_df["QT"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

param_grids = {
    "rf": {
        "n_estimators": [100, 200, 300],
        "max_depth": [10, 15, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["sqrt", "log2"],
    },
    "xgb": {
        "n_estimators": [100, 200],
        "max_depth": [3, 4],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2],
        "learning_rate": [0.01, 0.05],
    },
}

qt_models = {
    "rf": GridSearchCV(
        RandomForestRegressor(random_state=42),
        param_grids["rf"],
        cv=5,
        n_jobs=-1,
        scoring="r2",
    ),
    "xgb": GridSearchCV(
        estimator=GradientBoostingRegressor(random_state=42),
        param_grid=param_grids["xgb"],
        scoring="r2",
        cv=5,
        n_jobs=-1,
        verbose=1,
    ),
}

In [32]:
y_pred = np.clip(qt_model.predict(X_test), 0, 10)

r2 = r2_score(y_test, y_pred)
print(f"R^2 score for predicting TH: {r2}")

# Plotting actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    "r--",
    lw=2,
    label="Perfect Prediction",
)
plt.title("Actual vs. Predicted QT Scores")
plt.xlabel("Actual QT Score")
plt.ylabel("Predicted QT Score")
plt.legend()
plt.grid(True)
plt.show()

In [34]:
X = extracted_wecode_df[qt_feature_cols]
y_pred_qt = np.clip(qt_model.predict(X), 0, 10)
predictions_df = pd.DataFrame(
    {"username": extracted_wecode_df["username"], "Predicted_QT": y_pred_qt}
)
predictions_df.to_csv(f"{OUTPUT_DIR}qt_predictions.csv", index=False)

### 4.2. Huấn luyện mô hình dự đoán điểm thực hành (TH)

In [35]:
th_feature_cols = feature_cols


X = data_df[th_feature_cols]
y = data_df["TH"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [36]:
th_model = LinearRegression()
th_model.fit(X_train, y_train)

In [37]:
y_pred = np.clip(th_model.predict(X_test), 0, 10)

r2 = r2_score(y_test, y_pred)
print(f"R^2 score for predicting TH: {r2}")

# Plotting actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot(
    [y_test.min(), y_test.max()],
    [y_test.min(), y_test.max()],
    "r--",
    lw=2,
    label="Perfect Prediction",
)
plt.title("Actual vs. Predicted TH Scores")
plt.xlabel("Actual TH Score")
plt.ylabel("Predicted TH Score")
plt.legend()
plt.grid(True)
plt.show()

In [38]:
X = extracted_wecode_df[th_feature_cols]
y_pred_th = np.clip(th_model.predict(X), 0, 10)
predictions_df = pd.DataFrame(
    {"username": extracted_wecode_df["username"], "predicted_TH": y_pred_th}
)
predictions_df.to_csv(f"{OUTPUT_DIR}th_predictions.csv", index=False)
print("Predictions saved to th_predictions.csv")

### 4.3. Huấn luyện mô hình dự đoán điểm cuối kì (CK)